<a href="https://colab.research.google.com/github/FahimS45/Python_mini_projects/blob/master/Time_Frequency_Analysis_of_Wavelets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **A Project Regarding Time-Frequency Analysis of EEG data**

In [ ]:
# Importing all necessary modules
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat

# **Creating real and complex Morlet wavelets**

**Real Morlet Wavelets**

In [ ]:
# Functions to create Real wavelets
def createRealWavelet(time,freq,fwhm):
  # time = time for wavelet, should be zero-centered
  # freq = peak frequency for wavelet
  # fwhm = full-width at half-maximum in seconds
  sinepart = np.cos(2*np.pi*freq*time)
  # sinepart(t) = cos(2πft)
  gauspart = np.exp( (-4*np.log(2)*time**2)/(fwhm**2) )
  # gauspart(t) = e^(-(4ln(2)t^2) / (σ^2))
  return sinepart*gauspart

In [ ]:
# Parameters
freq  = 5 # Hz
fwhm  = .5
srate = 500 # Hz #sampling rate
time  = np.arange(-2*srate,2*srate)/srate
npnts = len(time) # number of time points

# Creating one wavelet and visualizing in time and in frequency domains
wavelet = createRealWavelet(time,freq,fwhm)

hz = np.linspace(0,srate/2,int(npnts/2))
# An array of frequency values from 0 to half of the sampling rate (srate/2), evenly spaced.

waveletX = abs(np.fft.fft(wavelet)/npnts)**2
# The power spectrum of the wavelet.

In [ ]:
# Setting-up the figure
fig,ax = plt.subplots(1,2,figsize=(15,5))

# Time-domain version
ax[0].plot(time,wavelet,'k')
ax[0].set_xlabel('Time (s)')
ax[0].set_ylabel('Amplitude (a.u.)')
ax[0].set_title('Time domain')

# Frequency-domain version
ax[1].stem(hz,waveletX[:len(hz)],'k',use_line_collection=True)
ax[1].plot(hz,waveletX[:len(hz)],'m')
ax[1].set_xlim([0,20])
ax[1].set_xlabel('Time (s)')
ax[1].set_ylabel('Amplitude (a.u.)')
ax[1].set_title('Frequency domain')

plt.show()

**Complex-valued Morlet wavelets**

In [ ]:
# Function to create Complex wavelets
def createComplexWavelet(time,freq,fwhm):
  sinepart = np.exp( 1j*2*np.pi*freq*time )
  gauspart = np.exp( (-4*np.log(2)*time**2)/(fwhm**2) )
  return sinepart*gauspart

In [ ]:
# Creating a complex Morlet wavelet
wavelet = createComplexWavelet(time,5,1)

# Plotting using matplotlib
fig = plt.subplots(1,figsize=(15,8))
plt.plot(time,np.real(wavelet),label='Real part')
plt.plot(time,np.imag(wavelet),label='Imaginary part')
plt.plot(time,np.abs(wavelet),'k',label='Magnitude')

plt.xlabel('Time (s)')
plt.legend(fontsize=19)
plt.show()

In [ ]:
# Plotting its magnitude and phase using matplotlib
fig = plt.subplots(1,figsize=(15,8))
plt.plot(time,np.angle(wavelet),label='Phase')
plt.plot(time,np.abs(wavelet),'k',label='Magnitude')

plt.xlabel('Time (s)')
plt.ylabel('Angle (rad.) or amplitude (a.u.)')
plt.legend()
plt.show()

# **Let's create a wavelet family**

In [ ]:
# Parameters
nfrex  =   40
# number of frequency values we want in the range between lofreq and hifreq

lofreq =    2   # Hz
hifreq =   80   # Hz

frex   = np.linspace(lofreq,hifreq,nfrex)
print(frex)
fwhms  = np.linspace(4,1,nfrex)
print(fwhms)

In [ ]:
# Creating a family of wavelets
waveletfam = np.zeros((nfrex,npnts),dtype=complex)
print(waveletfam)
print(npnts)

for wi in range(nfrex):
  waveletfam[wi,:] = createComplexWavelet(time,frex[wi],fwhms[wi])
print(waveletfam)

In [ ]:
# Setting-up the figure
fig,ax = plt.subplots(1,3,figsize=(15,5))

# Showing the real part
ax[0].imshow(np.real(waveletfam),
             aspect='auto',origin='lower',
             extent=[time[0],time[-1],lofreq,hifreq],
             vmin=-.8,vmax=.8)
# The extent parameter in imshow is used to define the range of values for
# the x-axis and y-axis of the plot. It takes a list or tuple of
# four values: [x_min, x_max, y_min, y_max].

ax[0].set_xlabel('Time (s)')
ax[0].set_ylabel('Frequency (Hz)')
ax[0].set_title('Real part')

# Showing the angles
ax[1].imshow(np.angle(waveletfam),
             aspect='auto',origin='lower',
             extent=[time[0],time[-1],lofreq,hifreq])

ax[1].set_xlabel('Time (s)')
ax[1].set_ylabel('Frequency (Hz)')
ax[1].set_title('Phase')


# Showing the magnitudes
ax[2].imshow(np.abs(waveletfam),
             aspect='auto',origin='lower',
             extent=[time[0],time[-1],lofreq,hifreq])

ax[2].set_xlabel('Time (s)')
ax[2].set_ylabel('Frequency (Hz)')
ax[2].set_title('Magnitude')


plt.show()

# **Importing the EEG data from a local drive**

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
# Importing the data to python
from scipy.io import loadmat
EEG = loadmat('sampleEEGdata.mat')
#print(EEG)

# Extracting the necessary information
times = np.squeeze(EEG['EEG'][0][0][14])
# np.squeeze is used to remove any extra dimensions from the extracted data.

data  = EEG['EEG'][0][0][15]
fs    = EEG['EEG'][0][0][11][0][0].astype(int) # sampling rate

print(fs)
print(np.shape(data)) # The size of the data array along each dimension.

In [ ]:
# computing Event-Related Potential (ERP)

erp = np.mean(data,axis=2)[46,:]
# Computes the mean along the third axis
# This results in an array where each element represents
# the average value across trials for a specific time point.

# Plotting trial-averaged response usning matplotlib
plt.plot(times,erp)
plt.xlim([-200,1000])
plt.xlabel('Time (ms)')
plt.ylabel('Voltage ($\mu V$)')
plt.title('ERP from channel 47')
plt.show()